In [13]:
import yfinance as yf

import os
os.chdir('C:\\Users\\btindol\\OneDrive - Stryker\\Linked In Learn\\Algorithmic Investing\Algorithmic Trading & Quantitative Analysis using Python')  
print(os.getcwd())  # Prints the current working directory

#####################################################################################################################################################
#imports 
#!pip install yfinance 
#!pip install pandas 
#!pip install numpy
#!pip install requests
!pip install datetime
#!pip install yahoofinancials
!pip install stocktrends
from yahoofinancials import YahooFinancials

import pandas as pd
import numpy as np
import requests
import datetime as dt
import os
import json 
from functools import reduce
import statsmodels.api as sm
from stocktrends import Renko
import pandas as pd
import numpy as np
import requests
import datetime as dt


import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\btindol\OneDrive - Stryker\Linked In Learn\Algorithmic Investing\Algorithmic Trading & Quantitative Analysis using Python


You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


# SINGLE STOCK ANALYZER NOW MAKE STRATEGY OFF OF IT

In [83]:
stocks = ['ETH-USD']
end_date = (dt.date.today()).strftime('%Y-%m-%d')
beg_date = (dt.date.today()-dt.timedelta(1825)).strftime('%Y-%m-%d') # 1825 is 5 years ish 


# Used to make a dataframe of percent change from the daily price data!!!
def get_daily_returns(df):
    daily_return = df.pct_change()
    cols = daily_return.columns.tolist()
    newcols = []
    for j in range(0,len(cols)):
        colz = cols[j] + "_" + "percent_change"

        newcols.append(colz)
    daily_return.columns = newcols               
    return daily_return

# Calculating RSI without using loop
def rsi(df, n):
    "function to calculate RSI"
    delta = df.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n])                                            # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n])                                             # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    return 100 - 100 / (1+rs)

def ATR(n):
    DF = yf.download(ticker,beg_date,end_date)[["Open","High","Low","Adj Close","Volume"]]
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    #df['ATR'] = df['TR'].rolling(n).mean()
    df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean() # 14 periods exponential weighted moving average to measure longer volitility use 90 day
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2[['Open','High','Low','ATR','TR']]

def ADX(DF,n):
    "function to calculate ADX"
    df2 = DF.copy()
    df2['TR'] = ATR(n)['TR'] #the period parameter of ATR function does not matter because period does not influence TR calculation
    df2['DMplus']=np.where((df2['High']-df2['High'].shift(1))>(df2['Low'].shift(1)-df2['Low']),df2['High']-df2['High'].shift(1),0)
    df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
    df2['DMminus']=np.where((df2['Low'].shift(1)-df2['Low'])>(df2['High']-df2['High'].shift(1)),df2['Low'].shift(1)-df2['Low'],0)
    df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
    TRn = []
    DMplusN = []
    DMminusN = []
    TR = df2['TR'].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i < n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
            DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
            DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
        elif i > n:
            TRn.append(TRn[i-1] - (TRn[i-1]/n) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/n) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/n) + DMminus[i])
    df2['TRn'] = np.array(TRn)
    df2['DMplusN'] = np.array(DMplusN)
    df2['DMminusN'] = np.array(DMminusN)
    df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
    df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
    df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
    df2['DIsum']=df2['DIplusN']+df2['DIminusN']
    df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
    ADX = []
    DX = df2['DX'].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2['DX'][j-n+1:j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
    df2['ADX']=np.array(ADX)
    return df2['ADX']


def MACD(DF,a,b,c):
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    df = DF.copy()
    df["MA_Fast"]=final_f.iloc[:,0].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=final_f.iloc[:,0].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return df[['MACD','Signal']]


def single_stock_analyzer(stocks, begin_date,end_date,initial_position,slow_ma,fast_ma):
    cl_price = pd.DataFrame()

    # looping over tickers and creating a dataframe with closing prices
    for ticker in stocks:
        cl_price[ticker] = yf.download(ticker,beg_date,end_date)['Adj Close']     # get just adjusted close column

    cl_price_plus_ret = get_daily_returns(cl_price)
    final = cl_price.merge(cl_price_plus_ret, on='Date', how='left')
    
    #########################################################################################
    #########################################################################################
    # for each of the column names for the % change columns 
    pctchg = [col for col in cl_price_plus_ret.columns if 'percent_change' in col] # grabe % change columns
    colnew = []                                                                    # initiate empty list
    for i in range(0,len(pctchg)):                                                 # loop over column names
        print(pctchg[i])
        var1 = cl_price_plus_ret[pctchg[i]].shift(-1)                                      # grab colunmn by name and shift up one
        colnew.append(var1)                                                        # append to list
    colnew = pd.DataFrame(colnew)                                                  # now its wide dataframe
    colnew = colnew.transpose()                                                    # make it narrow
    #########################################################################################
    #########################################################################################
    cols = colnew.columns.tolist()                                                 # take column names of narrow dataframe                                                
    newcols = []                                                                   # make list to add the text shift to the column name
    for j in range(0,len(cols)): 
        colz = cols[j] + "_" + "shift"                                             # add shift to the name of shifted column
           
        newcols.append(colz) 
    colnew.columns = newcols                                                       # assign the new shifted columns to origional
    
    final_f= final.merge(colnew, on='Date', how='left')

    final_f =final_f.fillna(0)     
    # fill na
    final_f.columns = final_f.columns.str.replace("-", "_")                         # replace all innapropriate characthers with  underscores in column names
    
    #########################################################################################
    #########################################################################################
    # Calculate returns from initial position entry 
    shiftval = [col for col in final_f.columns if 'shift' in col]                   # find the shift column to bring % change up to today to mutliply to get positions next day amount.. 
    
    initial_position = initial_position                                             # pass from function above
    
    new = [(final_f[shiftval[0]][0] * initial_position) + initial_position]        # get first value in final for iterate the 0 of this 
    for i in range(1, len(final_f.index)):                                          # starting at second row of the dataframe column 
        new.append((new[i-1]*final_f[shiftval].values[i][0]) +new[i-1])

    final_f['Position'] = new
    final_f['Position'] = final_f['Position'].shift(1)
    final_f['Position'][0] = initial_position
    final_f.fillna(0)
    
    #########################################################################################
    #########################################################################################
    # Try to figure out how to call out by name here... for multiple stocks 
    final_f['Fast_moving_average'] = final_f.iloc[:,0].rolling(window=fast_ma).mean() # 5 day price moving average
    final_f['Slow_moving_average'] = final_f.iloc[:,0].rolling(window=slow_ma).mean() # 5 day price moving average

    #########################################################################################
    #########################################################################################
    # Bolligner band of the fast moving average 30 day.. 
    final_f["BB_up"] = final_f["Fast_moving_average"] + 2*final_f.iloc[:,0].rolling(fast_ma).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    final_f["BB_dn"] = final_f["Fast_moving_average"] - 2*final_f.iloc[:,0].rolling(fast_ma).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    final_f["BB_width"] = final_f["BB_up"] - final_f["BB_dn"]
    
    #########################################################################################
    #########################################################################################
    # RSI
    # What Is the Relative Strength Index (RSI)?
    # The relative strength index (RSI) is a momentum indicator used in technical analysis that
    #measures the magnitude of recent price changes to evaluate overbought or oversold conditions in
    #the price of a stock or other asset. 
    #The RSI is displayed as an oscillator (a line graph that moves between two extremes) and can have a
    #reading from 0 to 100. The indicator was originally developed by J. Welles Wilder Jr. and introduced in his seminal 1978 book,
    #"New Concepts in Technical Trading Systems."
    # Traditional interpretation and usage of the RSI are that values of 70 or above indicate that a security is becoming overbought
    #or overvalued and may be primed for a trend reversal or corrective pullback in price. An RSI reading of 30 or below indicates
    #an oversold or undervalued condition.
    final_f["RSI_slow"] = rsi(final_f.iloc[:,0],slow_ma) # look at last values using slow as window
    final_f["RSI_fast"] = rsi(final_f.iloc[:,0],fast_ma)
    
    #########################################################################################
    #########################################################################################
    # ATR 
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #  To measure recent volatility, use a shorter average, such as 2 to 10 periods. For longer-term volatility, use 20 to 50 periods.
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/atr
    #Average True Range (ATR) is the average of true ranges over the specified period.
    #ATR measures volatility, taking into account any gaps in the price movement. 
    #Typically, the ATR calculation is based on 14 periods, which can be intraday, daily, weekly, or monthly. 
    #To measure recent volatility, use a shorter average, such as 2 to 10 periods. 
    #For longer-term volatility, use 20 to 50 periods.

    ATR2 = ATR(fast_ma)
        
    final_fz = final_f.merge(ATR2, on='Date', how='left')
    
    
    #########################################################################################
    #########################################################################################
    # ADX average directional movement 
    #Wilder suggests that a strong trend is present when ADX is above 25 and no trend is present when below 20.
    #If the ADX is declining, it could be an indication that the market is becoming less directional, and the current trend is weakening. You may want to avoid trading trend systems as the trend changes.
    #If after staying low for a lengthy time, the ADX rises by 4 or 5 units, (for example, from 15 to 20), it may be giving a signal to trade the current trend.
    # ADX AVERAGE DIRECTIONAL MOVEMENT
    #https://www.investopedia.com/articles/trading/07/adx-trend-indicator.asp
    # ADX values help traders identify the strongest and most profitable trends to trade. 
    #The values are also important for distinguishing between trending and non-trending conditions. 
    #Many traders will use ADX readings above 25 to suggest that the trend is strong enough for trend-trading strategies. 
    #Conversely, when ADX is below 25, many will avoid trend-trading strategies.

    # ADX Value	Trend Strength
    # 0-25	Absent or Weak Trend
    # 25-50	Strong Trend
    # 50-75	Very Strong Trend
    # 75-100	Extremely Strong Trend
    
    final_fz['ADX'] = ADX(final_fz,slow_ma)
    
    #########################################################################################
    #########################################################################################
    # Moving average convergence and divergence
    #The Formula for MACD Is:
    #MACD=12-Period EMA − 26-Period EMA

    # KEY TAKEAWAYS
    # Moving average convergence divergence (MACD) is calculated by subtracting the 26-period exponential moving average (EMA) from the 12-period EMA.
    # MACD triggers technical signals when it crosses above (to buy) or below (to sell) its signal line.
    # The speed of crossovers is also taken as a signal of a market is overbought or oversold.
    # MACD helps investors understand whether the bullish or bearish movement in the price is strengthening or weakening.

    # use typeical mcad values for now can change them later 
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    a = 12;b=26;c=9
    MACD2 = MACD(final_fz, 12, 26, 9)
    
    
    final_fz = final_f.merge(MACD2, on='Date', how='left')
    return final_fz




In [84]:
final_f = single_stock_analyzer(stocks,beg_date,end_date,10.506600,30,90)
final_f

[*********************100%***********************]  1 of 1 completed
ETH-USD_percent_change
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,ETH_USD,ETH_USD_percent_change,ETH_USD_percent_change_shift,Position,Fast_moving_average,Slow_moving_average,BB_up,BB_dn,BB_width,RSI_slow,RSI_fast,MACD,Signal
Date,,,,,,,,,,,,,
2016-05-13,10.506600,0.000000,-0.025403,10.506600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-14,10.239700,-0.025403,-0.027086,10.239700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-15,9.962350,-0.027086,0.121352,9.962350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-16,11.171300,0.121352,0.091977,11.171300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-17,12.198800,0.091977,0.111470,12.198800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-07,3484.729004,-0.001762,0.119929,3484.728878,2043.469047,2585.728288,3008.879976,1078.058118,1930.821857,70.765898,65.985009,341.155349,263.121725
2021-05-08,3902.647705,0.119929,0.006713,3902.647564,2068.895935,2646.197420,3105.654145,1032.137725,2073.516421,74.595807,68.270808,379.782124,286.453805
2021-05-09,3928.844727,0.006713,0.005968,3928.844584,2093.142912,2708.088615,3198.426263,987.859560,2210.566703,74.809571,68.405375,407.807056,310.724455
